In [12]:
import pandas as pd
import os
from datetime import datetime
from constants import PATH_STATS

SEP = ";"

In [7]:
os.listdir(os.path.join(PATH_STATS, "webscraping")), \
os.listdir(os.path.join(PATH_STATS, "webscraping", "processes_2023 09 21"))

(['processes_2023 09 20', 'processes_2023 09 21', 'processes_2023 09 24'],
 ['process_2023 09 21_23 13 20_concat.csv',
  'process_2023 09 21_23 13 20_pid_summary.csv',
  'process_2023 09 21_23 51 14_concat.csv',
  'process_2023 09 21_23 51 14_pid_summary.csv'])

In [8]:
# datetime.today().strftime("%Y %m %d")
# datetime.today().isoformat(sep=" ", timespec="hours")
datetime.today().strftime("%Y %m %d")

'2023 09 24'

In [28]:
files_same_date = []
date = input("Write a date with format 'year month day': ")
try:
    date_parts = date.split(" ")
    y, m, d = date_parts
    if date_parts != 3 or (len(y) != 4 or len(m) != 2 or len(d) != 2):
        date = datetime.today().strftime("%Y %m %d")
except:
    date = datetime.today().strftime("%Y %m %d")
time = input("Write a time with format 'hours minutes seconds': ")
try:
    time_parts = time.split(" ")
    h, m, s = time_parts
    if time_parts != 3 or (len(h) != 2 or len(m) != 2 or len(s) != 2):
        on_last_folder = True
except:
    on_last_folder = True
for folder in os.listdir(os.path.join(PATH_STATS, "webscraping")):
    if date in folder:
        process_folder = folder
        break

main_path = os.path.join(PATH_STATS, "webscraping", process_folder)
if on_last_folder:
    complete_path = os.path.join(main_path, os.listdir(main_path)[-1])
    stats_files = os.listdir(complete_path)
else:
    complete_path = os.path.join(main_path, time)
    stats_files = os.listdir(complete_path)
stats_files

['process_2023 09 24_15 52 01_pid_13256.csv',
 'process_2023 09 24_15 52 01_pid_15828.csv',
 'process_2023 09 24_15 52 01_pid_17020.csv',
 'process_2023 09 24_15 52 01_pid_2564.csv',
 'process_2023 09 24_15 52 01_pid_2676.csv',
 'process_2023 09 24_15 52 01_pid_2804.csv',
 'process_2023 09 24_15 52 01_pid_3056.csv',
 'process_2023 09 24_15 52 01_pid_3744.csv',
 'process_2023 09 24_15 52 01_pid_4652.csv',
 'process_2023 09 24_15 52 01_pid_6188.csv',
 'process_2023 09 24_15 52 01_pid_summary.csv']

In [29]:
df = []
for file in stats_files:
    if "summary" not in file:
        df.append(pd.read_csv(os.path.join(complete_path, file), 
                              sep=SEP,
                              names=["media_url",
                                     "n_urls",
                                     "n_body_gpt",
                                     "duration"
                                     ]
                                     )
                                     )
df = pd.concat(df, axis=0).iloc[1:]
df = df.astype({"media_url": "str", 
                "n_urls": "int", 
                "n_body_gpt": "int", 
                "duration": "float"})

df.to_csv(os.path.join(complete_path, 
                       "process_" + date + "_concat.csv"))

for file in os.listdir(os.path.join(complete_path)):
    if "summary" not in file and "concat" not in file:
        os.remove(os.path.join(complete_path, file))

print(df.shape)
df.head()

(54, 4)


,media_url,n_urls,n_body_gpt,duration
1,https://www.larioja.com/,0,0,0.3
2,https://www.ideal.es/,1,0,0.8
3,https://www.diariodemallorca.es/,0,0,0.4
0,https://www.ultimahora.es/,0,0,0.2
1,https://www.lavanguardia.com/,2,0,1.4


In [30]:
counts = df.filter(regex="n_").sum()
counts, df["duration"].max() / 60

(n_urls        49
 n_body_gpt     0
 dtype: int64,
 0.060000000000000005)

In [31]:
df_summary = pd.read_csv(os.path.join(complete_path, stats_files[-1]), 
                                   sep=SEP,
                                   header=None,
                                   names=["media_url",
                                          "n_urls",
                                          "n_body_gpt",
                                          "duration"
                                          ])
df_summary = df_summary.iloc[1:]
df_summary = df_summary.astype({"media_url": "str", 
                "n_urls": "int", 
                "n_body_gpt": "int", 
                "duration": "float"})
df_summary.head()

,media_url,n_urls,n_body_gpt,duration
1,Process summary,7,0,50.4
2,Process summary,1,0,50.8
3,Process summary,5,0,51.9
4,Process summary,6,0,52.5
5,Process summary,7,0,52.8


In [32]:
counts_summ = df_summary.filter(regex="n_").sum()
counts_summ, df_summary["duration"].max() / 60

(n_urls        41
 n_body_gpt     0
 dtype: int64,
 1.0450000000000002)